In [ ]:
from app import fetch_data

authors = fetch_data()

In [ ]:
for author in authors:
    for funder in author.funders:
        print(f'{author.name} was funded by {funder.name}')

In [ ]:
from supabase import create_client, Client
import os

# Load .env file if it exists
from dotenv import load_dotenv
from sqlalchemy import text
from sqlalchemy.orm import Session, selectinload
from extensions import session_factory
from models import Author, Funder, Institution, Paper, AuthorFunders
from schema import Author as AuthorSchema, Paper as PaperSchema
import json
from crud import update_database


load_dotenv()


# Replace with your Supabase project URL and API key
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")

# Create a Supabase client
supabase: Client = create_client(supabase_url, supabase_key)


def fuzzy_match_author(session: Session, name: str, threshold=0.7):
    print("Threshold:", threshold)  # Print the threshold value
    query = text("SELECT * FROM authors WHERE similarity(name, :name) > :threshold")
    result = session.execute(
        query, {"name": name, "threshold": threshold}
    ).fetchall()
    if result:
        author = (
            session.query(Author)
            .filter_by(id=result[0].id)
            .options(
                selectinload(Author.funders), selectinload(Author.institutions)
            )
            .first()
        )
        return author
    return None


def fuzzy_match_funder(session: Session, name: str, threshold=0.7):
    print("Threshold:", threshold)  # Print the threshold value
    query = text("SELECT * FROM funders WHERE similarity(name, :name) > :threshold")
    result = session.execute(
        query, {"name": name, "threshold": threshold}
    ).fetchall()
    if result:
        funder = session.query(Funder).filter_by(id=result[0].id).first()
        return funder


def fuzzy_match_institution(session: Session, name: str, threshold=0.7):
    print("Threshold:", threshold)  # Print the threshold value
    query = text(
        "SELECT * FROM institutions WHERE similarity(name, :name) > :threshold"
    )
    result = session.execute(
        query, {"name": name, "threshold": threshold}
    ).fetchall()
    if result:
        institution = session.query(Institution).filter_by(id=result[0].id).first()
        return institution
    return None


def update_database(paper: PaperSchema, author: AuthorSchema):
    with session_factory() as session:
        session: Session

        # DOI lookup of paper
        db_paper = session.query(Paper).filter_by(doi=paper.doi).first()
        if not db_paper:
            db_paper = Paper(doi=paper.doi)
            session.add(db_paper)
            session.commit()
            session.refresh(db_paper)

        # Fuzzy lookup of author
        db_author = fuzzy_match_author(session, author.name)
        if not db_author:
            db_author = Author(name=author.name)
            session.add(db_author)
            print("Author not found, adding new author to the database")
        session.commit()

        # Process each funder for the author
        for funder_schema in author.funders:
            db_funder = fuzzy_match_funder(session, funder_schema.name)
            if not db_funder:
                db_funder = Funder(name=funder_schema.name)
                session.add(db_funder)
            
            session.commit()  # Ensure db_funder is persisted to get an ID if newly created
            
            # Check if the AuthorFunders association already exists
            existing_association = session.query(AuthorFunders).filter_by(
                author_id=db_author.id, funder_id=db_funder.id, paper_id=db_paper.id
            ).first()

            # If the association doesn't exist, create it
            if not existing_association:
                new_association = AuthorFunders(
                    author_id=db_author.id, funder_id=db_funder.id, paper_id=db_paper.id
                )
                session.add(new_association)
                print(f"Linking {db_author.name}, {db_funder.name}, and paper {db_paper.doi}")

        session.commit()

        # Fuzzy match institutions
        db_institutions = []
        for institution in author.institution:
            institution: Institution
            db_institution = fuzzy_match_institution(session, institution.name)
            if not db_institution:
                db_institution = Institution(name=institution.name)
                session.add(db_institution)
                print("Institution not found, adding new institution to the database")
            else:
                # db_institution = session.merge(db_institution)
                print(f"Institution {db_institution} found in the database")
            db_institutions.append(db_institution)

        # Update author with matched institutions and funders
        for institution in db_institutions:
            if institution not in db_author.institutions:
                db_author.institutions.append(institution)
                print(f"Adding {institution} to {db_author}")

        session.commit()


        # Update paper with matched authors and funders
        if db_author not in db_paper.authors:
            db_paper.authors.append(db_author)
        for funder in paper.funders:
            db_funder = fuzzy_match_funder(session, funder.name)
            if not db_funder:
                db_funder = Funder(name=funder.name)
                session.add(db_funder)
                print("Funder for Paper not found, adding new funder to the database")
            else:
                db_funder = session.merge(db_funder)
                print(f"Funder {db_funder} found in the database")
            if db_funder not in db_paper.funders:
                db_paper.funders.append(db_funder)

        session.add(db_paper, db_author)
        session.commit()
        session.close()

# Load the JSON data
with open("real3.json") as file:
    json_data = json.load(file)

# Parse the JSON data into Pydantic models
authors = [AuthorSchema(**author_data) for author_data in json_data[0]["authors"]]
paper = PaperSchema(doi=json_data[0]["doi"], authors=authors, funders=json_data[0]["funders"])
# Update the database
for author in authors:
    update_database(paper, author)

In [ ]:
from app2 import fetch_author_via_paper_doi

authors = fetch_author_via_paper_doi("https://doi.org/10.1016/j.jinf.2024.106130")
for author in authors:
    print(f"Author: {author.name}")
    for funder in author.funders:
        print(f"Funder: {funder.name}")